<a href="https://colab.research.google.com/github/willmbennett/meld_calc/blob/dev/mimic_iv_data_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## MIMIC IV Data Cleaning
The purpose of this notebook is to access the [MIMIC-IV](https://physionet.org/content/mimiciv/2.2/) files stored in BigQuery, clean them, and turn them into a csv for reading. Before doing this it's necessary to complete the steps to access the data.

To do this it's neccesary to authenticate

In [1]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


I'll also mount Google Drive to store the final data

In [2]:
# GDRIVE local files
from google.colab import drive
drive.mount('/content/drive')
import sys
sys.path.append('/content/drive/MyDrive/Flatiron/phase 5/colab')

Mounted at /content/drive


In [3]:
# Load the data_table module
%load_ext google.colab.data_table

In [4]:
%%bigquery df --project mimic-iv-388714

-- Stary by pulling all ICD codes from diagnosis
WITH dx AS
(
  SELECT hadm_id, icd_version, TRIM(icd_code) AS icd_code
  FROM `physionet-data.mimiciv_hosp.diagnoses_icd`
)

-- ICD 9 codes to include
, dx_icd9 AS
(
	SELECT hadm_id,
        MAX(CASE
            WHEN icd_code IN ('5715', '5712', '5716') THEN 1
        ELSE 0 END) AS explicit_cirrhosis
        ,MAX(CASE
            WHEN icd_code IN ('30390', 'V113', '30500', '5715', '5718', '30503', '5711', '30393', '5712',
            '30391', '30501', '5710', '30301', '9808', '30302', '5713', '3575', '30300', 'V6141', '5719',
            '30392', '53530', '2911', '29181', '30502', '2910', '29189', '53531', '2913', '9800', '9802',
            '2919', '4255', '30303', '9809', '2912', '9773', 'E8603', '76071', 'E8609', 'E9473', 'E8600',
            '29182', '7903') THEN 1
        ELSE 0 END) AS alcohol_use
        ,MAX(CASE
            WHEN icd_code IN ('5718', '5728', '5719') THEN 1
        ELSE 0 END) AS liver_disease
        ,MAX(CASE
            WHEN icd_code IN ('57142', '7032', '7022', '7033') THEN 1
        ELSE 0 END) AS hep
	FROM dx
    WHERE icd_version = 9
    GROUP BY hadm_id
),

-- ICD 10 codes to include
dx_icd10 AS
(
    SELECT
        hadm_id,
        MAX(CASE
            WHEN icd_code IN ('K7030', 'K7460', 'K7031', 'K7469', 'K743', 'K717', 'K744', 'K745') THEN 1
        ELSE 0 END) AS explicit_cirrhosis
        ,MAX(CASE
            WHEN icd_code IN ('F1021', 'K7581', 'K7040', 'K7030', 'K7010', 'F1011', 'Z811', 'F1010', 'F1020',
            'F10129', 'K7031', 'K700', 'F1096', 'K7011', 'Y906', 'Y900', 'F10229', 'Y908', 'F10929', 'Y905',
            'K2920', 'Y909', 'G621', 'F1097', 'Y904', 'Y901', 'K709', 'F10220', 'Y907', 'F10120', 'F10239',
            'F10231', 'F1014', 'K8520', 'F1094', 'F10121', 'K852', 'F10230', 'F1024', 'K860', 'K8521', 'F10221',
            'F1027', 'G312', 'F1026', 'F10232', 'F10259', 'K2921', 'K8522', 'T518X2A', 'T5194XA', 'K7041',
            'F10959', 'F10250', 'F10951', 'F1099', 'F1019', 'F10251', 'O99311', 'F1029', 'F10920', 'F10988',
            'I426', 'F10188', 'F10288', 'T5192XA', 'F10151', 'O99313', 'F10980', 'O99312', 'Y903', 'F10180',
            'F10159', 'G721', 'K702', 'R780', 'Q860', 'O99314', 'E244', 'Y902', 'T5191XA', 'F10280', 'Z7141',
            'Z6372', 'F10282') THEN 1
        ELSE 0 END) AS alcohol_use
        ,MAX(CASE
            WHEN icd_code IN ('K769', 'K713', 'K7589', 'K7110', 'K712', 'K759', 'K709', 'K716', 'K710', 'K719',
            'K717', 'K7151', 'K718', 'K7111') THEN 1
        ELSE 0 END) AS liver_disease
        ,MAX(CASE
            WHEN icd_code IN ('K8301') THEN 1
        ELSE 0 END) AS psc
        ,MAX(CASE
            WHEN icd_code IN ('K754', 'B182', 'B181', 'B189', 'B188') THEN 1
        ELSE 0 END) AS hep
    FROM dx
    WHERE icd_version = 10
    GROUP BY hadm_id
)

-- Liver Transplant from procedure data (will be excluded)
, proc_icd9 as
(
	SELECT hadm_id,
		MAX(CASE
		WHEN TRIM(icd_code) IN ('V427', '99682') THEN 1
		ELSE 0 END) AS liv_trans
	FROM `physionet-data.mimiciv_hosp.procedures_icd`
    WHERE icd_version = 9
    GROUP BY hadm_id
)
, proc_icd10 AS
(
	SELECT hadm_id,
		MAX(CASE
		WHEN TRIM(icd_code) IN ('T8641', 'T8649', 'T8642', 'T8643', 'Z944', 'Z4823', 'T8640') THEN 1
		ELSE 0 END) AS liv_trans
	FROM `physionet-data.mimiciv_hosp.procedures_icd`
    WHERE icd_version = 10
    GROUP BY hadm_id
)
-- Aggregate above views together
, aggregate as
(
	SELECT adm.subject_id, adm.hadm_id
		, GREATEST(COALESCE(dx_icd9.explicit_cirrhosis, 0), COALESCE(dx_icd10.explicit_cirrhosis, 0)) AS explicit_cirrhosis
		, GREATEST(COALESCE(proc_icd9.liv_trans, 0), COALESCE(proc_icd10.liv_trans, 0)) AS liv_trans
		, GREATEST(COALESCE(dx_icd9.alcohol_use, 0), COALESCE(dx_icd10.alcohol_use, 0)) AS alcohol_use
		, GREATEST(COALESCE(dx_icd9.liver_disease, 0), COALESCE(dx_icd10.liver_disease, 0)) AS liver_disease
		, COALESCE(dx_icd10.psc, 0) AS psc
		, GREATEST(COALESCE(dx_icd9.hep, 0), COALESCE(dx_icd10.hep, 0)) AS hep
	FROM `physionet-data.mimiciv_hosp.admissions` adm
    LEFT JOIN dx_icd9
        ON adm.hadm_id = dx_icd9.hadm_id
    LEFT JOIN dx_icd10
        ON adm.hadm_id = dx_icd10.hadm_id
    LEFT JOIN proc_icd9
        ON adm.hadm_id = proc_icd9.hadm_id
    LEFT JOIN proc_icd10
        ON adm.hadm_id = proc_icd10.hadm_id
)

, cohort_creation as (
-- Output component flags (explicit_cirrhosis, alcohol_use, liver_disease, psc, hep) and final flag (cirrhosis)
SELECT subject_id, hadm_id, explicit_cirrhosis, alcohol_use, liver_disease, psc, hep,
COALESCE(GREATEST(explicit_cirrhosis, alcohol_use, liver_disease, psc, hep), 0) AS cirrhosis
FROM aggregate
WHERE liv_trans != 1 -- remove patients admitied for liver transplant or related
  AND (explicit_cirrhosis = 1
        OR alcohol_use = 1
        OR liver_disease = 1
        OR psc = 1
        OR hep = 1)
)

-- Final table creation
SELECT ie.stay_id
       ,id.admittime
       ,id.dod
       ,id.race
       ,id.gender
       ,id.admission_age as age
       ,labs.*
       ,r.dialysis_present
       ,m.meld_initial
       ,m.meld
       ,case when id.dod <= CAST(DATETIME_ADD(id.admittime, INTERVAL 90 DAY) AS DATE) then 1 else 0 end as target
FROM `physionet-data.mimiciv_icu.icustays` ie
JOIN `physionet-data.mimiciv_derived.icustay_detail` id ON id.stay_id = ie.stay_id
JOIN `physionet-data.mimiciv_derived.meld` m on m.stay_id =  ie.stay_id
JOIN cohort_creation cohort on cohort.hadm_id = ie.hadm_id
LEFT JOIN `physionet-data.mimiciv_derived.first_day_lab` labs ON ie.stay_id = labs.stay_id
LEFT JOIN `physionet-data.mimiciv_derived.first_day_rrt` r ON ie.stay_id = r.stay_id

Query is running:   0%|          |

Downloading:   0%|          |

Now that I've queried the data I will check out the data

In [5]:
df.head(5)

,stay_id,admittime,dod,race,gender,age,subject_id,stay_id_1,hematocrit_min,hematocrit_max,...,ck_mb_min,ck_mb_max,ggt_min,ggt_max,ld_ldh_min,ld_ldh_max,dialysis_present,meld_initial,meld,target
0,30908451,2136-06-28 19:23:00,NaT,WHITE,M,36,10122346,30908451,32.4,35.5,...,5.0,5.0,NaN,NaN,NaN,NaN,<NA>,6.000000000,6.0,0
1,38120799,2113-03-08 10:38:00,2113-03-12,WHITE,M,55,16942323,38120799,24.5,26.7,...,NaN,NaN,76.0,76.0,733.0,733.0,<NA>,16.000000000,16.0,1
2,35807915,2130-01-18 18:46:00,2130-01-21,WHITE,M,42,14576761,35807915,22.6,32.5,...,18.0,20.0,NaN,NaN,1586.0,1861.0,1,40.000000000,40.0,1
3,34920278,2126-06-10 19:57:00,2126-06-24,UNKNOWN,M,81,17403434,34920278,26.2,38.5,...,NaN,NaN,NaN,NaN,180.0,239.0,<NA>,29.000000000,29.0,1
4,35217267,2113-12-13 18:31:00,2113-12-25,OTHER,F,67,10098875,35217267,22.7,23.0,...,NaN,NaN,NaN,NaN,304.0,304.0,<NA>,11.000000000,11.0,1


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11381 entries, 0 to 11380
Data columns (total 98 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   stay_id                 11381 non-null  Int64         
 1   admittime               11381 non-null  datetime64[ns]
 2   dod                     4683 non-null   dbdate        
 3   race                    11381 non-null  object        
 4   gender                  11381 non-null  object        
 5   age                     11381 non-null  Int64         
 6   subject_id              11381 non-null  Int64         
 7   stay_id_1               11381 non-null  Int64         
 8   hematocrit_min          11292 non-null  float64       
 9   hematocrit_max          11292 non-null  float64       
 10  hemoglobin_min          11281 non-null  float64       
 11  hemoglobin_max          11281 non-null  float64       
 12  platelets_min           11283 non-null  float6

**Observations**
- There are a significant number of empty columns
- There are a lot of features (98), it would be good to reduce these

I will start by dropping unneeded columns

In [7]:
df_clean = df.drop(['admittime','dod', 'meld_initial', 'stay_id', 'subject_id', 'stay_id_1'], axis=1) # drop date of death and time of admission

I will drop columns with < 60% of data

In [8]:
thresh = len(df_clean)*0.6
thresh

6828.599999999999

In [9]:
df_clean.dropna(axis=1, thresh=thresh, inplace=True)

Convert columns with incorrect types to floats

In [10]:
cols_to_convert = list(df_clean.select_dtypes(object).drop(['race','gender'], axis=1).columns)

for col in cols_to_convert:
  df_clean[col] = df_clean.loc[:, col].astype(float)

I will investigate & clean the key categorical columns race and sex

In [28]:
df_clean['race'].value_counts()

WHITE                                        7135
UNKNOWN                                      1248
BLACK/AFRICAN AMERICAN                        996
OTHER                                         359
HISPANIC/LATINO - PUERTO RICAN                195
WHITE - OTHER EUROPEAN                        165
HISPANIC OR LATINO                            164
UNABLE TO OBTAIN                              160
PORTUGUESE                                    110
ASIAN                                          94
HISPANIC/LATINO - DOMINICAN                    94
ASIAN - CHINESE                                81
BLACK/CAPE VERDEAN                             65
WHITE - RUSSIAN                                51
HISPANIC/LATINO - GUATEMALAN                   51
PATIENT DECLINED TO ANSWER                     50
BLACK/AFRICAN                                  49
AMERICAN INDIAN/ALASKA NATIVE                  41
ASIAN - SOUTH EAST ASIAN                       40
BLACK/CARIBBEAN ISLAND                         39


In [29]:
def clean_race_column(race):
  if race in ('OTHER', 'UNKNOWN', 'UNABLE TO OBTAIN', 'PATIENT DECLINED TO ANSWER'):
    return 'UNKNOWN'
  else:
    return race

In [30]:
df_clean['race'] = df_clean.loc[:,'race'].apply(clean_race_column)

In [31]:
df_clean['race'].value_counts()

WHITE                                        7135
UNKNOWN                                      1817
BLACK/AFRICAN AMERICAN                        996
HISPANIC/LATINO - PUERTO RICAN                195
WHITE - OTHER EUROPEAN                        165
HISPANIC OR LATINO                            164
PORTUGUESE                                    110
ASIAN                                          94
HISPANIC/LATINO - DOMINICAN                    94
ASIAN - CHINESE                                81
BLACK/CAPE VERDEAN                             65
HISPANIC/LATINO - GUATEMALAN                   51
WHITE - RUSSIAN                                51
BLACK/AFRICAN                                  49
AMERICAN INDIAN/ALASKA NATIVE                  41
ASIAN - SOUTH EAST ASIAN                       40
BLACK/CARIBBEAN ISLAND                         39
HISPANIC/LATINO - SALVADORAN                   37
WHITE - BRAZILIAN                              29
NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER      22


Now I will check out gender

In [33]:
df_clean['gender'].value_counts()

M    7869
F    3512
Name: gender, dtype: int64

That looks clean, I'm ready to save the file

In [32]:
# Save cleaned data file
df_clean.to_csv('mimic_iv_cleaned.csv', index=False)
!cp mimic_iv_cleaned.csv "drive/My Drive/Flatiron/phase_5/colab/data"